**Purpose:** Notes on good ideas for organizing and developing a Python project.

## Project layout

Use `src`-layout, see [src-layout](https://setuptools.pypa.io/en/latest/userguide/package_discovery.html#src-layout).

In [1]:
%%sh
mkdir -p src/foo

## Virtual environment

Use a virtual environment.

In [2]:
%%sh
python -m venv venv
source venv/bin/activate

## Editable install

Then we install the package in editable mode, see [editable-installs](https://pip.pypa.io/en/latest/cli/pip_install/#editable-installs) and [development-mode](https://setuptools.readthedocs.io/en/latest/userguide/quickstart.html#development-mode). This enables us to edit the package on the fly without the need to reinstall it.

The `-e` flag installs the module in editable mode, meaning that you can modify your source code in `foo/` and have the changes take effect without you having to rebuild and reinstall, see [3].

```bash
~$ pip install -h
-e, --editable <path/url>   Install a project in editable mode (i.e. setuptools "develop mode") from a local project path or a VCS url.
```

To run an editable install we need a `setup.py` file.

In [3]:
%%writefile setup.py

from setuptools import setup, find_packages


setup(
    name="foo",
    version="0.0.1",
    description="A package illustrating editable install.",
    url="https://github.com/laegsgaardTroels/autoreload",
    package_dir={"": "src"},
    packages=find_packages(
        where="src",
        include=["foo*"],
    ),
)

Overwriting setup.py


In [4]:
%%sh
pip install -e .

Obtaining file:///Users/troelslaegsgaard/Git/laegsgaardTroels/laegsgaardTroels.github.io/src/posts/2024-03-03-python-project
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Attempting uninstall: foo
    Found existing installation: foo 0.0.1
    Uninstalling foo-0.0.1:
      Successfully uninstalled foo-0.0.1
  Running setup.py develop for foo



[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


## Jupyter magic

Use `autoreload` when making prototypes, see [autoreload](https://ipython.org/ipython-doc/3/config/extensions/autoreload.html).

`autoreload` is an IPython extension to reload modules before executing user code.

`autoreload` reloads modules automatically before entering the execution of code typed at the IPython prompt.

This makes for example the following workflow possible:

In [5]:
%load_ext autoreload
%autoreload 2

In [6]:
%%writefile src/foo/mappings.py

def some_function() -> float:
    return 42 

Overwriting src/foo/mappings.py


In [7]:
from foo import mappings

mappings.some_function()

42

Now if we change `some_function` to return 43.

In [8]:
%%writefile src/foo/mappings.py

def some_function() -> float:
    return 43

Overwriting src/foo/mappings.py


And we call it again then.

In [9]:
mappings.some_function()

43

The module was reloaded without reloading it explicitly, and the object imported with `from foo import some_function` was also updated.

## Testing

Often used tools for testing in Python are `pytest` (most common) and `unittest`.

So use `pytest` and put the tests in a `tests/` folder in the same directory. This is recommended by `pytest`, see [tests-outside-application-code](https://docs.pytest.org/en/7.1.x/explanation/goodpractices.html#tests-outside-application-code)

In [10]:
%%sh
mkdir -p tests

## Type Checking

Often used are `pyright`.

## Code style formatter

Often used are `flake8` or for automatic code formatting people often use `black`.

## Pre-commit hooks

[pre-commit](https://pre-commit.com) is often used to run a series of checks before making commits to git e.g. `black` code formatting, `pyright`, `isort` etc.

## Logging

Logging is a nice debugging tool in Python. <!--more--> A mimimal example would be:

In [11]:
%%writefile src/foo/downloader.py
import logging

FILES = ['foo', 'bar']

logger = logging.getLogger(__name__)

  
def download_all():
    for filename in FILES:
        download_file(filename)


def download_file(filename):
    logger.info(f'Beginning download of {filename}')

Overwriting src/foo/downloader.py


The loggers are instantiated using `logging.getLogger(__name__)` in each script. You can then configure the log in a main script using `logging.basicConfig(...)`.

In [12]:
import logging 
import sys

logging.basicConfig(
    format='%(asctime)s - %(message)s',
    stream=sys.stdout,
    level=logging.INFO,
    datefmt='%d-%b-%y %H:%M:%S',
    force=True
)

In [13]:
from foo import downloader

downloader.download_all()

01-Apr-24 19:21:45 - Beginning download of foo
01-Apr-24 19:21:45 - Beginning download of bar


## Cookiecutter

[cookiecutter](https://github.com/cookiecutter/cookiecutter) can be used to create a template Python project.